In [ ]:
import json
import os
import boto3
from datetime import datetime
from tmdbv3api import TMDb, Discover, Movie


tmdb = TMDb()
tmdb.api_key = os.environ.get('API_KEY') 
tmdb.language = 'pt-BR'  
discover = Discover()
movie_api = Movie()


s3_client = boto3.client('s3')


def buscar_filmes_animacao():
    genero_animacao_id = 16
    ano_limite = 2022
    lista_filmes = []

    for pagina in range(1, 21):  
        resultados = discover.discover_movies({
            'with_genres': genero_animacao_id,
            'release_date.lte': f'{ano_limite}-12-31',
            'sort_by': 'popularity.desc',
            'page': pagina
        })
        if not resultados:
            break

        for filme in resultados:
            
            detalhes_filme = movie_api.details(filme.id)

       
            lista_filmes.append({
                'id': filme.id,
                'titulo': filme.title,
                'ano_lancamento': filme.release_date.split('-')[0] if filme.release_date else None,
                'estudio': detalhes_filme.production_companies[0]['name'] if detalhes_filme.production_companies else None,
                'genero': [genero['name'] for genero in detalhes_filme.genres] if detalhes_filme.genres else None,
                'diretores': [membro['name'] for membro in detalhes_filme.credits['crew'] if membro['job'] == 'Director'] if 'credits' in detalhes_filme and 'crew' in detalhes_filme.credits else None,
                'orcamento': detalhes_filme.budget,
                'arrecadacao': detalhes_filme.revenue
            })

    return lista_filmes


def salvar_no_s3(dados, bucket_name):
    try:
        tamanho_chunk = 100 
        contador_arquivo = 0
        data_atual = datetime.now()
        caminho_base = f"Raw/TMDB/JSON/{data_atual.year}/{data_atual.month:02}/{data_atual.day:02}/"

        for i in range(0, len(dados), tamanho_chunk):
            chunk = dados[i:i + tamanho_chunk]
            nome_arquivo = f"file_{contador_arquivo}.json"
            caminho_completo = f"{caminho_base}{nome_arquivo}"

            
            json_data = json.dumps(chunk, ensure_ascii=False, indent=4)

           
            s3_client.put_object(Bucket=bucket_name, Key=caminho_completo, Body=json_data, ContentType='application/json')

            print(f"Dados salvos com sucesso no S3: {caminho_completo}")
            contador_arquivo += 1

    except Exception as e:
        print(f"Erro ao salvar os dados no S3: {e}")

# Função principal para Lambda
def lambda_handler(event, context):
    bucket_name = "data-lake-mateus"  
    filmes = buscar_filmes_animacao()
    if filmes:
        salvar_no_s3(filmes, bucket_name)
    return {
        'statusCode': 200,
        'body': json.dumps('Processamento concluído com sucesso!')
    }
